In [1]:
import dash
from dash import html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd



external_stylesheets = ['style.css']
app = dash.Dash(__name__ ,suppress_callback_exceptions=True,external_stylesheets=external_stylesheets)

# Shivam

In [2]:
import pickle
with open('Shivam/genre_count_year_movies.pkl', 'rb') as file:
    genre_count_year_movies = pickle.load(file)

with open('Shivam/movies_dict.pkl', 'rb') as file:
    movies_dict = pickle.load(file)

with open('Shivam/genre_year_scatter_figures_dict.pkl', 'rb') as file:
    genre_year_scatter_figures_dict = pickle.load(file)

with open('Shivam/movies.pkl', 'rb') as file:
    movies = pickle.load(file)

with open('Shivam/fig_movies.pkl', 'rb') as file:
    fig_movies = pickle.load(file)

with open('Shivam/tv_series_revenue_df.pkl', 'rb') as file:
    tv_series_revenue_df = pickle.load(file)

with open('Shivam/similarity_content.pkl', 'rb') as file:
    similarity_content = pickle.load(file)
    
with open('Shivam/tv_series_dict.pkl', 'rb') as file:
    tv_series_dict = pickle.load(file)

In [3]:
def getGenreOverYearsScatter(release_year):
    counts_per_year = release_year.value_counts().sort_index().to_frame()
    counts_per_year.reset_index(inplace=True)
    # Filter out movies released before 1980
    counts_per_year = counts_per_year[counts_per_year['Release Year'] >= 1980]
    fig = go.Figure(data = go.Scatter(x = counts_per_year['Release Year'], y = counts_per_year['count']))
    # Update layout
    fig.update_layout(title='TV Shows Count Per Year',
                  xaxis_title='Year',
                  yaxis_title='TV Shows Count')
    return fig
# Dictionary to store figures for each genre
genre_year_scatter_figures_dict = {}

# Iterate through each genre DataFrame in the tv_series_dict dictionary
for genre, df in tv_series_dict.items():
    # Run the function for the current genre DataFrame and store the resulting figure
    genre_year_scatter_figures_dict[genre] = getGenreOverYearsScatter(df['Release Year'])

# Dictionary to store figures for each genre for movies
genre_year_scatter_figures_dict_movies = {}

# Iterate through each genre DataFrame in the movies_dict dictionary
for genre, df in movies_dict.items():
    # Run the function for the current genre DataFrame and store the resulting figure
    genre_year_scatter_figures_dict_movies[genre] = getGenreOverYearsScatter(df['Release Year'])


@app.callback(
    Output('genre-charts_SS', 'figure'),
    [Input('genre-dropdown_SS', 'value')]
)
def update_genre_charts(selected_genres):
    # Create empty figure with subplots
    fig = make_subplots(rows=1, cols=2, subplot_titles=('TV Shows Count Per Year', 'Movies Count Per Year'))

    # Iterate through selected genres
    for genre in selected_genres:
        # Add traces for TV shows
        trace_tv = genre_year_scatter_figures_dict[genre]['data'][0]  # Assuming only one trace per figure
        trace_tv['name'] = genre
        # Increase the width of markers
        trace_tv['marker']['size'] = 15
        fig.add_trace(trace_tv, row=1, col=1)

        # Add traces for movies
        trace_movies = genre_year_scatter_figures_dict_movies[genre]['data'][0]  # Assuming only one trace per figure
        trace_movies['name'] = genre
        # Increase the width of markers
        trace_movies['marker']['size'] = 15
        fig.add_trace(trace_movies, row=1, col=2)

    # Update layout
    fig.update_layout(height=600, width=1000, showlegend=False)

    return fig

def getGenreOverYearsBar(dictionary):
    # Define the range of years
    years = list(range(1980, 2022))
    count_year_wise_dict = {str(year): {} for year in years}
    color_list = ['blue', 'green', 'red', 'orange', 'purple', 'yellow', 'cyan', 'magenta', 'lime', 'pink']
    # Iterate over the years and populate the dictionary
    for year in years:
        for genre in dictionary.keys():
            count_year_wise_dict[str(year)][genre] = {}
    genre_count_year = {}
    for year in years:
        genre_count = {}
        for genre,df in dictionary.items():
            genre_count[genre] =  len(df[df['Release Year']==year])
        temp_df = pd.DataFrame.from_dict(genre_count,orient='index').reset_index()
        temp_df.columns = ['Genre','Count']

        # Assign colors to each genre
        num_colors = len(color_list)
        color_index = 0
        genre_colors = {}

        for genre in temp_df['Genre'].unique():
            genre_colors[genre] = color_list[color_index]
            color_index = (color_index + 1) % num_colors

        # Map colors to genres
        temp_df['Color'] = temp_df['Genre'].map(genre_colors)
        genre_count_year[str(year)] = temp_df
        genre_count_year[str(year)] = genre_count_year[str(year)].sort_values(by='Count')
    return genre_count_year

def getGenreOverYearsBarMovies(dictionary):
    # Define the range of years
    years = list(range(1980, 2022))
    count_year_wise_dict = {str(year): {} for year in years}
    color_list = ['blue', 'green', 'red', 'orange', 'purple', 'yellow', 'cyan', 'magenta', 'lime', 'pink']
    # Iterate over the years and populate the dictionary
    for year in years:
        for genre in dictionary.keys():
            count_year_wise_dict[str(year)][genre] = {}
    genre_count_year = {}
    for year in years:
        genre_count = {}
        for genre, df in dictionary.items():
            genre_count[genre] = len(df[df['Release Year'] == year])
        temp_df = pd.DataFrame.from_dict(genre_count, orient='index').reset_index()
        temp_df.columns = ['Genre', 'Count']

        # Assign colors to each genre
        num_colors = len(color_list)
        color_index = 0
        genre_colors = {}

        for genre in temp_df['Genre'].unique():
            genre_colors[genre] = color_list[color_index]
            color_index = (color_index + 1) % num_colors

        # Map colors to genres
        temp_df['Color'] = temp_df['Genre'].map(genre_colors)
        genre_count_year[str(year)] = temp_df
        genre_count_year[str(year)] = genre_count_year[str(year)].sort_values(by='Count')
    return genre_count_year

def update_violin_chart(selected_genre):
    # Filter data for TV shows
    tv_data = tv_series_dict[selected_genre]
    tv_data = tv_data[tv_data['Certificate'].isin(["R", "PG-13", "TV-MA", "TV-14", "PG"])]

    # Filter data for movies
    movie_data = movies_dict[selected_genre]  # Assuming you have a dictionary 'movie_dict' with movie data
    movie_data = movie_data[movie_data['Certificate'].isin(["R", "PG-13", "TV-MA", "TV-14", "PG"])]

    # Create subplots
    fig = make_subplots(rows=2, cols=1, subplot_titles=("TV Shows", "Movies"))

    # Add violin chart for TV shows
    fig.add_trace(
        go.Violin(y=tv_data['Rating'], x=tv_data['Certificate'], box_visible=True, line_color='black',
                  meanline_visible=True, fillcolor='pink', opacity=0.6, name='Rating Distribution - TV Shows'),
        row=1, col=1
    )

    # Add violin chart for movies
    fig.add_trace(
        go.Violin(y=movie_data['Rating'], x=movie_data['Certificate'], box_visible=True, line_color='black',
                  meanline_visible=True, fillcolor='lightblue', opacity=0.6, name='Rating Distribution - Movies'),
        row=2, col=1
    )

    # Update layout
    fig.update_layout(title=f"Rating Distribution for {selected_genre}",
                      xaxis_title='Certificate',
                      yaxis_title='Rating',
                        height=800)

    return fig

@app.callback(
    Output('violin-chart_SS2', 'figure'),
    [Input('genre-dropdown_SS2', 'value')]
)
def update_chart(selected_genre):
    return update_violin_chart(selected_genre)

# Define function to get genre rating line chart
def getGenreRatingLineChart(selected_genres):
    fig = go.Figure()
    for genre in selected_genres:
        data = tv_series_dict[genre]
        data = data[data['Release Year'] != 2023]
        yearly_total_rating_df = data.groupby('Release Year')['Rating'].sum().reset_index()

        # Add trace for the genre
        fig.add_trace(go.Scatter(x=yearly_total_rating_df['Release Year'], y=yearly_total_rating_df['Rating'], mode='lines', name=genre))

    # Update layout
    fig.update_layout(
        title='Total Rating Over Years',
        xaxis_title='Year',
        yaxis_title='Total Rating'
    )

    return fig

@app.callback(
    Output('rating-line-chart_SS4', 'figure'),
    [Input('genre-dropdown_SS4', 'value')]
)
def update_rating_line_chart(selected_genres):
    return getGenreRatingLineChart(selected_genres)


def getGenreRatingLineChart(selected_genres):
    fig = go.Figure()
    for genre in selected_genres:
        data = tv_series_dict[genre]
        # Remove year 2023
        yearly_median_rating_df = data.groupby('Release Year')['Rating'].median().reset_index()

        # Add trace for the genre
        fig.add_trace(go.Scatter(x=yearly_median_rating_df['Release Year'], y=yearly_median_rating_df['Rating'], mode='lines', name=genre))

    # Update layout
    fig.update_layout(
        title='Median Rating of TV Shows Per Year',
        xaxis_title='Year',
        yaxis_title='Median Rating'
    )

    return fig

@app.callback(
    Output('rating-line-chart_SS5', 'figure'),
    [Input('genre-dropdown_SS5', 'value')]
)
def update_rating_line_chart(selected_genres):
    return getGenreRatingLineChart(selected_genres)

# YearWise how rating is affected-Genre Wise
def getYearGenreRatingArea(selected_genre):
    data = tv_series_dict[selected_genre]
    # Filter data based on selected certificates
    filtered_data = data[data['Certificate'].isin(["R","PG-13","TV-MA","TV-14","PG"])]


    # Group by year and certificate, calculate average rating
    certificate_ratings = filtered_data.groupby(['Release Year', 'Certificate'])['Rating'].median().unstack()

    # Delete Year which have NaN as a rating
    certificate_ratings = certificate_ratings.dropna()
    certificate_ratings = certificate_ratings.reset_index()

    # Create a color list with a visually distinct palette
    color_palette = ['firebrick', 'royalblue', 'forestgreen', 'orange', 'purple']

    # Create a Trace for each certificate
    fig = make_subplots(rows=3, cols=2, vertical_spacing=0.05)
    for i, certificate in enumerate(["R","PG-13","TV-MA","TV-14","PG"]):
        trace = go.Scatter(
            x=certificate_ratings['Release Year'],
            y=certificate_ratings[certificate],
            name=certificate,
            mode='lines+markers',  # Include both lines and data points
            line=dict(color=color_palette[i%5], width=2),  # Consistent line style
            marker=dict(size=8, symbol='circle'),  # Clearer markers
        )
        fig.append_trace(trace, row=i//2 + 1, col=i%2 + 1)
    fig.update_layout(
        title="Average Rating by Release Year (Selected Certificates)",
        height=800,
        xaxis_title=dict(text='Rating', font=dict(size=16))
    )
    return fig

# Define callback to update the chart based on selected genre
@app.callback(
    Output('rating-chart_SS6', 'figure'),
    [Input('genre-dropdown_SS6', 'value')]
)
def update_chart(selected_genre):
    return getYearGenreRatingArea(selected_genre)

# Define function to get genre runtime line chart
def getGenreRuntimeLineChart(selected_genres):
    fig = go.Figure()
    for genre in selected_genres:
        data = tv_series_dict[genre]
        # Remove year 2023
        yearly_median_runtime_df = data.groupby('Release Year')['Runtime'].median().reset_index()

        # Add trace for the genre
        fig.add_trace(go.Scatter(x=yearly_median_runtime_df['Release Year'], y=yearly_median_runtime_df['Runtime'], mode='lines', name=genre))

    # Update layout
    fig.update_layout(
        title='Median Runtime of TV Shows Per Year',
        xaxis_title='Year',
        yaxis_title='Median Runtime'
    )

    return fig

@app.callback(
    Output('runtime-line-chart_SS7', 'figure'),
    [Input('genre-dropdown_SS7', 'value')]
)
def update_runtime_line_chart(selected_genres):
    return getGenreRuntimeLineChart(selected_genres)



# **Vipul and Vishnu Tab**

In [4]:
import pickle
with open('Vipul/figV1.pkl', 'rb') as file:
    figV1 = pickle.load(file)

with open('Vipul/time_traces.pkl', 'rb') as file:
    time_traces = pickle.load(file)

with open('Vipul/df_top_critic_pntve.pkl', 'rb') as file:
    df_top_critic_pntve = pickle.load(file)

with open('Vipul/top_actors_by_genre.pkl', 'rb') as file:
    top_actors_by_genre = pickle.load(file)

with open('Vipul/dictionary_top_5_directors_each_genre.pkl', 'rb') as file:
    dictionary_top_5_directors_each_genre = pickle.load(file)

with open('Vipul/indices_top_5_directors_each_genre.pkl', 'rb') as file:
    indices_top_5_directors_each_genre = pickle.load(file)

with open('Vipul/df_top5_productions.pkl', 'rb') as file:
    df_top5_productions = pickle.load(file)

with open('Vipul/df_content.pkl', 'rb') as file:
    df_content = pickle.load(file)



genrelist = ['Action & Adventure', 'Comedy', 'Drama', 'Science Fiction & Fantasy', 'Romance', 'Classics', 'Kids & Family', 'Mystery & Suspense', 'Western', 'Art House & International', 'Horror', 'Faith & Spirituality', 'Animation', 'Documentary', 'Special Interest', 'Musical & Performing Arts', 'Sports & Fitness', 'Television', 'Cult Movies', 'Anime & Manga', 'Gay & Lesbian', '']

@app.callback(
    Output('genre-line-chart', 'figure'),
    [Input('genre-dropdown', 'value')]
)
def update_line_chart(selected_genres):
    traces = []
    for genre in selected_genres:
        genre_df = time_traces[genre]
        trace = go.Scatter(
            x=genre_df['year'],
            y=genre_df['positive_reviews'],
            mode='lines',
            name=genre
        )
        traces.append(trace)
    layout = go.Layout(
        title={'text':'Positive Reviews Over Years', 'font': {'color': 'white'}},
        xaxis={'title': 'Year', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},  # Set x-axis font color to white
        yaxis={'title': 'Positive Reviews', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},  # Set y-axis font color to white
        hovermode='closest',
        plot_bgcolor='rgba(0,0,0,0)', 
        paper_bgcolor='rgba(0,0,0,0)',
        legend={'font': {'color': 'white'}}
    )
    return {'data': traces, 'layout': layout}

@app.callback(
    Output('genre-graph', 'figure'),
    [
        Input('genre-dropdown3', 'value'),
        Input('top-directors-slider', 'value'),
        Input('plot-type-radio', 'value')
    ]
)
def update_genre_graph(selected_genre, num_top_directors, plot_type):
    directors = dictionary_top_5_directors_each_genre[selected_genre + 'Ratio'][:num_top_directors]
    top_directors_pve = df_top_critic_pntve.loc[indices_top_5_directors_each_genre[selected_genre + 'Ratio'][:num_top_directors], selected_genre + 'P'].tolist()
    top_directors_nve = df_top_critic_pntve.loc[indices_top_5_directors_each_genre[selected_genre + 'Ratio'][:num_top_directors], selected_genre + 'N'].tolist()
    
    if plot_type == 'bar':
        fig = go.Figure(data=[
            go.Bar(name='Positive Reviews', x=directors, y=top_directors_pve,marker_color='#26D7FF'),
            go.Bar(name='Negative Reviews', x=directors, y=top_directors_nve,marker_color='#FFFFFF')
        ])
        fig.update_layout(
            title={'text':f'Reviews for Top {num_top_directors} Directors in {selected_genre}', 'font': {'color': 'white'}},
            barmode='group',
            xaxis={'title': 'Directors', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},
            yaxis={'title': 'Num of Reviews', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},
            plot_bgcolor='rgba(0,0,0,0)', 
            paper_bgcolor='rgba(0,0,0,0)',
            legend={'font': {'color': 'white'}}
        )
    elif plot_type == 'pie':

        color_start = np.array([0x08, 0x68, 0xB9])  # RGB values for #26D7FF
        color_end = np.array([0xFF, 0xFF, 0xFF])    # RGB values for #FFFFFF
        num_slices = len(directors)
        colors = [(color_start + (i / (num_slices - 1)) * (color_end - color_start)).astype(int).tolist() for i in range(num_slices)]
        colors = ['#%02x%02x%02x' % tuple(color) for color in colors]
        
        fig = go.Figure(data=[
            go.Pie(labels=directors, values=top_directors_pve, name='Positive Reviews', marker=dict(colors=colors)),
            go.Pie(labels=directors, values=top_directors_nve, name='Negative Reviews', marker=dict(colors=colors))
        ])
        
        
        fig.update_layout(
            title=f'Reviews Distribution for Top {num_top_directors} Directors in {selected_genre}',
            plot_bgcolor='rgba(0,0,0,0)', 
            paper_bgcolor='rgba(0,0,0,0)',
            legend={'font': {'color': 'white'}}
        )
        fig.update_traces(textposition='outside', textinfo='percent+label',textfont=dict(color='white'))

    
    return fig

@app.callback(
    Output('actor-bar-chart', 'figure'),
    [Input('genre-dropdown2', 'value')]
)
def update_bar_chart(selected_genre):
    # Get DataFrame of top actors for selected genre
    top_actors_df = top_actors_by_genre[selected_genre]
    
    # Create bar chart data
    data = [
        go.Bar(name='Positive Reviews', x=top_actors_df['actors'], y=top_actors_df['positive_reviews'], marker_color="#26D7FF"),
        go.Bar(name='Negative Reviews', x=top_actors_df['actors'], y=top_actors_df['negative_reviews'], marker_color="#FFFFFF")
    ]
    
    # Create bar chart layout with dynamic title
    layout = go.Layout(
        title={'text': f'Top 5 Actors in {selected_genre}', 'font': {'color': 'white'}},  # Dynamic title based on selected genre
        barmode='group',  # Display bars side by side
        xaxis={'title': 'Actors', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},
        yaxis={'title': 'Number of Reviews', 'tickfont': {'color': '#B4B4B4'},'titlefont': {'color': 'white'}, 'gridcolor': '#737373'},
        plot_bgcolor='rgba(0,0,0,0)', 
        paper_bgcolor='rgba(0,0,0,0)',
        legend={'font': {'color': 'white'}}
    )
    
    # Create bar chart figure
    figure = {'data': data, 'layout': layout}
    
    return figure

import numpy as np

@app.callback(
    Output('rating-pie-chart', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_pie_chart(selected_company):
    # Filter data for the selected production company
    selected_data = df_top5_productions[df_top5_productions['production_company'] == selected_company]
    
    # Extract ratings and corresponding values
    ratings = ['PG', 'R', 'NR', 'G', 'PG-13', 'NC17']
    values = [selected_data[rating].values[0] for rating in ratings]
    
    # Generate a range of colors from #26D7FF to #FFFFFF
    color_start = np.array([0x08, 0x68, 0xB9])  # RGB values for #26D7FF
    color_end = np.array([0xFF, 0xFF, 0xFF])    # RGB values for #FFFFFF
    num_slices = len(ratings)
    colors = [(color_start + (i / (num_slices - 1)) * (color_end - color_start)).astype(int).tolist() for i in range(num_slices)]
    colors = ['#%02x%02x%02x' % tuple(color) for color in colors]

    # Create pie chart using Plotly
    fig = go.Figure(data=[go.Pie(labels=ratings, values=values, marker=dict(colors=colors))])
    fig.update_layout(
        title={'text': f'Content Ratings Distribution for {selected_company}', 'font': {'color': 'white'}},
        plot_bgcolor='rgba(0,0,0,0)', 
        paper_bgcolor='rgba(0,0,0,0)', 
        legend={'font': {'color': 'white'}}
    )
    fig.update_traces(textposition='outside', textinfo='percent+label',textfont=dict(color='white'))
    
    return fig




@app.callback(
    Output('genre-treemapV', 'figure'),
    [Input('genre-dropdownV', 'value')]
)
def update_genre_treemap(selected_genre):
    selected_row = df_content[df_content['Genre'] == selected_genre]
    
    fig = px.treemap(
        selected_row.melt(id_vars='Genre', var_name='Rating', value_name='Count'),
        path=['Genre', 'Rating'],
        values='Count',
        color='Count',
        color_continuous_scale='ice',
        # title={'text':f'Treemap for {selected_genre}', 'font': {'color': 'white'}},
        title="hello"
    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', 
        paper_bgcolor='rgba(0,0,0,0)', legend={'font': {'color': 'white'}},
                     title={'text':f'Treemap for {selected_genre}', 'font': {'color': 'white'}})
    return fig



In [5]:
figV1.update_layout(
    xaxis=dict(showgrid=False, linecolor='black', tickfont=dict(color='white'),title=dict(font=dict(color='white'))),  # Remove vertical grid lines and set x-axis line color to black
    yaxis=dict(showgrid=True, gridcolor='black', tickfont=dict(color='white'),title=dict(font=dict(color='white'))),  # Show y-axis grid lines and set color to black
    title=dict(font=dict(color='white')),  # Set title font color to white
    legend=dict(font=dict(color='white')),  # Set legend font color to white
    sliders=[{
        'len': 0.9,  # Adjusts the length of the slider
        'x': 0.1,  # Positioning of the slider
        'y': 0,  # Positioning of the slider
        'active': 0,
        'currentvalue': {
            'xanchor': 'right',
            'prefix': 'Year:',
            'font': {'size': 20, 'color': 'white'}  # Set font color to white
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},  # Transition effect
        'bgcolor': '#26D7FF',  # Background color of the slider, set to a dark grey
        'borderwidth': 2,  # Border width of the slider
        'bordercolor': 'white',  # Border color of the slider
        'pad': {'b': 10, 't': 100},  # Padding
        'currentvalue': {'visible': True},  # Shows the current value
        'font': {'color': 'white'}  # Set slider font color to white
    }]
)

colors = ['#A3EEFF', '#26D7FF', '#FFFFFF', '#ECECEC']  # Define colors for each set of bars

for i, color in enumerate(colors):
    figV1.data[i].marker.color = color

for trace in figV1.data:
    trace.marker.line.color = 'rgba(0, 0, 0, 0)'  # Set border color to transparent


# Anshul and Ruby tab

In [6]:
with open('Anshul/A1.pkl', 'rb') as file:
    A1 = pickle.load(file)

with open('Anshul/A2.pkl', 'rb') as file:
    A2 = pickle.load(file)

with open('Anshul/A3.pkl', 'rb') as file:
    A3 = pickle.load(file)

with open('Anshul/A4.pkl', 'rb') as file:
    A4 = pickle.load(file)

with open('Anshul/A5.pkl', 'rb') as file:
    A5 = pickle.load(file)

with open('Anshul/A6.pkl', 'rb') as file:
    A6 = pickle.load(file)

with open('Anshul/A7.pkl', 'rb') as file:
    A7 = pickle.load(file)

with open('Anshul/A8.pkl', 'rb') as file:
    A8 = pickle.load(file)

with open('Anshul/A9.pkl', 'rb') as file:
    A9 = pickle.load(file)


#load pickle dataset for dash plot
    
#callback

# Web APP

In [7]:


app.layout = html.Div([
    dcc.Tabs(
        id='tabs',
        value='tab-1',
        children=[
            dcc.Tab(label='Tab 1', value='tab-1'),
            dcc.Tab(label='Anshul', value='tab-2'),
            dcc.Tab(label='Tab 3', value='tab-3'),
            dcc.Tab(label='Vipul', value='tab-4'),
        ],
          # Apply custom styling directly to the Tabs component
    ),
    html.Div(id='tabs-content')
], style={'background-image': 'linear-gradient(to right, black, darkgrey)',
                  'background-size': '400% 400%',}) # Setting background color directly in layout



@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H3('Tab 1 content'),
           dcc.Dropdown(
                id='genre-dropdown_SS',
                options=[
                    {'label': f'{genre}', 'value': genre} for genre in genre_year_scatter_figures_dict.keys()
                ],
                value=['Action'],  # Default value (multiple)
                multi=True,  # Allow multiple selections
                clearable=False
            ),
            dcc.Graph(id='genre-charts_SS'),
            dcc.Graph(
                id='fig_movies_SS',
                figure=fig_movies
            ),
            dcc.Dropdown(
                id='genre-dropdown_SS2',
                options=[
                    {'label': genre, 'value': genre} for genre in genre_year_scatter_figures_dict.keys()
                ],
                value='Action',  # Default value
                clearable=False
            ),
            dcc.Graph(id='violin-chart_SS2'),
            dcc.Dropdown(
                id='genre-dropdown_SS4',
                options=[{'label': genre, 'value': genre} for genre in tv_series_dict.keys()],
                value=['Action','Horror'],  # Default value
                multi=True,  # Allow multiple selections
                clearable=False
            ),
            dcc.Graph(id='rating-line-chart_SS4'),
            dcc.Dropdown(
                id='genre-dropdown_SS5',
                options=[{'label': genre, 'value': genre} for genre in tv_series_dict.keys()],
                value=['Action','Horror','Romance'],  # Default value
                multi=True,  # Allow multiple selections
                clearable=False
            ),
            dcc.Graph(id='rating-line-chart_SS5'),
            html.H1("TV Series Rating Distribution"),
            dcc.Dropdown(
                id='genre-dropdown_SS6',
                options=[{'label': genre, 'value': genre} for genre in tv_series_dict.keys()],
                value='Action'
            ),
            dcc.Graph(id='rating-chart_SS6'),
            dcc.Dropdown(
                id='genre-dropdown_SS7',
                options=[{'label': genre, 'value': genre} for genre in tv_series_dict.keys()],
                value=['Action','Horror','Romance'],  # Default value
                multi=True,  # Allow multiple selections
                clearable=False
            ),
            dcc.Graph(id='runtime-line-chart_SS7'),
            dcc.Slider(
                id='decade-slider_SS8',
                min=1980,
                max=2023,
                value=2021,  # Default value
                marks={year: f'{year}s' for year in range(1980, 2021, 10)},
                step=None,
                included=False # Trigger callback only on mouseup
            ),
            dcc.Graph(id='genre-runtime-bar_SS8'),
            dcc.Dropdown(
                id='genre-dropdown_SS9',
                options=[{'label': genre, 'value': genre} for genre in tv_series_revenue_df.keys()],
                value='Action'  # Default value
            ),
            dcc.Graph(id='scatter-plot_SS9')
            
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab 2 content'),
            html.Div([
                dcc.Graph(id='A1', 
                          figure=A1.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
                         ),
                dcc.Graph(
                    id='A2', 
                    figure=A2.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
                ),
            ], style={
                'display':'flex',
                'justify-content':'space-between',
                'flex-wrap':'wrap'
            } ),
            dcc.Graph(
                id='A3',
                figure=A3.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
            ),
            dcc.Graph(
                id='A4',
                figure=A4.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
            ),
            dcc.Graph(
                id='A5',
                figure=A5.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
            ),
            dcc.Graph(
                id='A6',
                figure=A6.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
            ),
            dcc.Graph(
                id='A8',
                figure=A8.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
            ),
            dcc.Graph(
                id='A9',
                figure=A9.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
            )
            
            
        ],style={'background-image': 'linear-gradient(to right, black, darkgrey)',
                  'background-size': '400% 400%',}    
        )
    elif tab == 'tab-3':
        return html.Div([
            html.H3('Tab 3 content'),
            dcc.Graph(
                id='tab3-graph',
                figure={
                    'data': [
                        {'labels': ['A', 'B', 'C'], 'values': [1, 2, 3], 'type': 'pie', 'name': 'Pie chart'},
                    ],
                    'layout': {
                        'title': 'Tab 3 Plot'
                    }
                }
            )
        ])
    elif tab == 'tab-4':
        return html.Div([
            html.H3('Tab 4 content'),
    
            # First graph
            html.Div([
                dcc.Graph(
                    id='tab4-graph',
                    figure=figV1.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')  # Assuming figV1 is defined elsewhere
                )
            ]),
    
            # Second graph with dropdown
            html.Div([
                dcc.Dropdown(
                    id='genre-dropdown',
                    options=[{'label': html.Span(genre), 'value': genre} for genre in time_traces.keys()],
                    value=['Animation'],  # Default selected genres
                    multi=True,  # Allow multiple selections
                    className='custom-dropdown',  # Add custom CSS class
                    style = {'background-color': '#000000'}
                ),
                dcc.Graph(id='genre-line-chart')
            ]),

    
            # Third graph with dropdown
            html.Div([
                dcc.Dropdown(
                    id='genre-dropdown2',
                    options=[{'label': html.Span(genre, style={'color': '#26D7FF'}), 'value': genre} for genre in top_actors_by_genre.keys()],
                    value=list(top_actors_by_genre.keys())[0],  # Default value for dropdown (first genre)
                    style={'width': '50%', 'background-color': 'rgba(0,0,0,0)', 'color': 'black'},
                    clearable=False,
                ),
                dcc.Graph(id='actor-bar-chart')
            ]),
    
            # Fourth graph with dropdown, slider, and radio items
            html.Div([
                dcc.Dropdown(
                    id='genre-dropdown3',
                    options=[{'label': html.Span(genre, style={'color': '#26D7FF'}), 'value': genre} for genre in genrelist],
                    value=genrelist[0],
                    clearable=False,
                    style={'width': '50%','background-color': 'rgba(0, 0, 0, 0)', 'color': '#000000'}
                ),
                dcc.Slider(
                    id='top-directors-slider',
                    min=1,
                    max=5,
                    step=1,
                    value=5,
                    marks={i: str(i) for i in range(1, 6)},
                ),
                dcc.RadioItems(
                    id='plot-type-radio',
                    options=[
                        {'label': 'Bar Chart', 'value': 'bar'},
                        {'label': 'Pie Chart', 'value': 'pie'}
                    ],
                    value='bar',
                    labelStyle={'display': 'block', 'background-color': 'rgba(0, 0, 0, 0)', 'color': '#26D7FF'}
                ),
                dcc.Graph(id='genre-graph')
            ],style={'background-color': 'rgba(0, 0, 0, 0)'}),
    
            # Fifth graph with dropdown
            html.Div([
                dcc.Dropdown(
                    id='company-dropdown',
                    options=[{'label': html.Span(company, style={'color': '#26D7FF'}), 'value': company} for company in df_top5_productions['production_company']],
                    value=df_top5_productions['production_company'][0],  # Default value is the first production company
                    style = {'background-color': 'rgba(0, 0, 0, 0)', 'color': 'black'},
                    clearable=False,
                ),
                dcc.Graph(id='rating-pie-chart')
            ]),
    
            # Sixth graph with dropdown
            html.Div([
                dcc.Dropdown(
                    id='genre-dropdownV',
                    options=[{'label': html.Span(genre, style={'color': '#26D7FF'}), 'value': genre} for genre in genrelist],
                    value=genrelist[0],
                    style={'width': '50%', 'background-color': 'rgba(0, 0, 0, 0)', 'color': 'black'},
                    clearable=False,
                ),
                dcc.Graph(id='genre-treemapV')
            ])
        ],style={'background-image': 'linear-gradient(to right, black, darkgrey)',
                  'background-size': '400% 400%',}  )




In [8]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8009)

# http://127.0.0.1:8009/
